In [21]:
import pandas as pd
import numpy as np
import os, itertools,pyodbc
import df_to_sql.df_to_sql as d2s
import df_to_sql.upin as up
from collections import defaultdict
from sqlalchemy import create_engine
from mysql import *
from dateutil.parser import *
from datetime import *
from dateutil.relativedelta import *


def modstr(strval):
    if isinstance(strval, str):
        try:
            s1 = strval.replace("'","\\'")
            return s1
        except:
            return strval
    else:
        return strval

def trc_dc(dic0, ky, vl):
    dic = defaultdict(list)
    dic = dic0.copy()
    dic[ky] = vl if not list(dic) else dic.get(ky, []) + [vl]
    return dic

def TM1(x):
    try:
        return parse(x)
    except:
        return x

class OMSQB:
    def __init__(self, table, cols=[]):
        self.T = table
        self.C = cols
        self.sel_ls = []
        self.ins = []
        self.upd_set = ''
        self.upd_set_ls = []
        self.cond = ''
        self.cond_ls = []
        self.track = defaultdict(dict)
        self.MT = ["M1. single_insert(self,vals=[])",
                   "M2. update_fault_test(self,cursors,vals,condcol,operator=' = ')",
                   "M3. update_set(self,vals,condcol=[],operator=' LIKE ', condop =' and ', enable_condcol=False)",
                   "M4. addcond(self, addby= ' OR ',condition ='')",
                   "M5. def q_select(self,col=False)",
                   "Var 1. self.track = defaultdict(dict)"
                  ]
        for x in self.MT:
            print(x)
        else:
            print(chr(10))
    def single_insert(self,vals=[]):
        VAL = ''
        cL = ''
        vL = ''
        for i2,v2 in enumerate(vals):
            if len(str(v2))>0:
                if isinstance(v2,str):
                    try:
                        VAL = "'" + v2.replace("'","\'") + "'"
                    except:
                        VAL = "'" + v2 + "'"
                else:
                    VAL = "'" + str(v2) + "'"
                if cL == '':
                    cL = self.C[i2]
                    vL = VAL
                else:
                    cL = cL + "," + self.C[i2]
                    vL = vL + "," + VAL
        else:
            insert_into = "INSERT INTO " + self.T + " (" + cL + ") VALUES (" + vL + ")"
            self.ins.append(insert_into)
            self.track = trc_dc(self.track,'single_insert',insert_into)
            return insert_into
    def update_fault_test(self,cursors,vals,condcol,operator=' = '):
        T0 = ' where ' + condcol + operator
        T1 = 'select * from ' + self.T + ' where ' + condcol + operator
        T2 = 'insert into ' + self.T + condcol + ' values '
        #cr = self.conn.cursor()
        for i,v in enumerate(vals):
            if self.C[i] == condcol:
                T0 = T0 + "'" + str(v) + "'"
                T1 = T1 + "'" + str(v) + "'"
                T2 = T2 + "'" + str(v) + "'"
                print('T0-T2 -- ',T0,T1,T2)
                try:
                    cursors.execute(T1)
                    print(T1)
                except:
                    cursors.execute(T2)
                    print(T2)
                finally:
                    break
        T31 = 'UPDATE ' + self.T + ' SET ' 
        for i,v in enumerate(vals):
            if self.C[i] != condcol:
                T32 = T31 + self.C[i] + "='" + str(v) + "'" + T0
                T42 = T31 + self.C[i] + "='" + str(TM1(v)) + "'" + T0
                try:
                    cursors.execute(T32)
                    print('pass: ', T32)
                except:
                    try:
                        cursors.execute(T42)
                        print('pass: ', T42)
                    except:
                        print('Failed: ', T32, " ~~ ", T42)
    def update_set(self,vals,condcol=[],operator=' LIKE ', condop =' and ', enable_condcol=False):
        vl = ''
        ccol = ''
        for i2,v2 in enumerate(vals):
            if self.C[i2] not in condcol and len(str(v2))>0:
                if vl == '':
                    vl = self.C[i2] + "='" + str(modstr(v2)) + "'"
                else:
                    vl = vl + "," + self.C[i2] + "='" + str(modstr(v2)) + "'"
            elif enable_condcol == True and self.C[i2] in condcol:
                tmp = self.C[i2] + operator + "'" + str(modstr(v2)) + "'"
                if ccol == '':
                    ccol = tmp
                else:
                    ccol = ccol + condop + tmp
        else:
            if ccol!= '':
                self.upd_set = 'UPDATE ' + self.T + ' SET ' + vl + ' WHERE ' + ccol
            elif condcol!= [] or enable_condcol==True:
                self.upd_set = 'UPDATE ' + self.T + ' SET ' + vl + ' WHERE '
            elif enable_condcol==False:
                self.upd_set = vl
            self.upd_set_ls.append(self.upd_set)
            self.track = trc_dc(self.track,'update_set',self.upd_set)
    def addcond(self, addby= ' OR ',condition =''):
        tmp = self.cond + addby + condition
        self.cond = tmp
        self.upd_set = self.upd_set + addby + condition
        self.cond_ls.append(condition)
        self.track = trc_dc(self.track,'addcond',condition)
    def q_select(self,col=False):
        sel = ''
        if col==False:
            sel = "select " + ','.join(str,self.C) + " from " + self.T
        else:
            if type(col) is list:
                sel = "select " + ','.join(str,col) + " from " + self.T
            else:
                sel = "select " + str(col) + " from " + self.T
        self.sel_ls.append(sel)
        self.track = trc_dc(self.track,'q_select',sel)
    def get_trc(self):
        return self.track
    def get_select(self):
        return self.sel_ls
    def get_ins(self):
        return self.ins
    def get_upd(self):
        return self.upd_set_ls
            
            
def mssql_conn(U='sa',P='1q2w3eaz$',H='192.168.0.102',D='omdb'):
    cstr2 = "mssql+pyodbc://" + U + ':' + P + '@' + H + "/" + D + "?driver=SQL Server"
    engin = create_engine(cstr2)
    conn = engin.raw_connection()
    return conn
def mssql_engin(U='sa',P='1q2w3eaz$',H='192.168.0.102',D='omdb'):
    cstr2 = "mssql+pyodbc://" + U + ':' + P + '@' + H + "/" + D + "?driver=SQL Server"
    engin = create_engine(cstr2)
    return engin


csv1 = "D:\\omEngin\\AOmPy_o_\\csv_o_\\sclick3.csv"
ndf = pd.read_csv(csv1)
col = ndf.columns.to_list()
val = ndf.values.tolist()
x = OMSQB('tbl7',col)
cn = mssql()
cr = cn.cursor()
x.update_fault_test(cr,val[6],condcol='Serial')


#x.update_set(val[4],['Serial','Resource'],enable_condcol=True)
#x.addcond(condition='Severity beteen 0 to 5')
#x.addcond(condition='type=1')
            


M1. single_insert(self,vals=[])
M2. update_fault_test(self,cursors,vals,condcol,operator=' = ')
M3. update_set(self,vals,condcol=[],operator=' LIKE ', condop =' and ', enable_condcol=False)
M4. addcond(self, addby= ' OR ',condition ='')
M5. def q_select(self,col=False)
Var 1. self.track = defaultdict(dict)




Exception during reset or similar
Traceback (most recent call last):
  File "e:\python\lib\site-packages\sqlalchemy\pool\base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "e:\python\lib\site-packages\sqlalchemy\pool\base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "e:\python\lib\site-packages\sqlalchemy\engine\default.py", line 543, in do_rollback
    dbapi_connection.rollback()
pyodbc.Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLEndTran); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')


T0-T2 --   where Serial = '855659573' select * from tbl7 where Serial = '855659573' insert into tbl7Serial values '855659573'
select * from tbl7 where Serial = '855659573'
pass:  UPDATE tbl7 SET Severity='Critical' where Serial = '855659573'
pass:  UPDATE tbl7 SET Node='DRE14' where Serial = '855659573'
pass:  UPDATE tbl7 SET Resource='DHMRP53UM' where Serial = '855659573'
Failed:  UPDATE tbl7 SET Incident Owner='FLS_RAN' where Serial = '855659573'  ~~  UPDATE tbl7 SET Incident Owner='FLS_RAN' where Serial = '855659573'
pass:  UPDATE tbl7 SET FirstOccurrence='2020-08-17 10:31:00' where Serial = '855659573'
pass:  UPDATE tbl7 SET LastOccurrence='2020-08-17 10:31:00' where Serial = '855659573'
pass:  UPDATE tbl7 SET Tally='1' where Serial = '855659573'
pass:  UPDATE tbl7 SET CustomAttr11='e.co_Nabinagar' where Serial = '855659573'
pass:  UPDATE tbl7 SET Summary='ERI-3G CELL DOWN' where Serial = '855659573'
pass:  UPDATE tbl7 SET AssociatedCR='nan' where Serial = '855659573'
pass:  UPDATE

In [15]:
 
    
from dateutil.parser import *
from datetime import *  

print(TS('17/8/2020 10:31'))

2020-08-17 10:31:00
